In [ ]:
####################################################################################
# Copyright 2025 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
####################################################################################


####################################################################################
# Demo for various preparation capabilities announced at Next25
#
# Author: Damien Contreras cdamien@google.com
####################################################################################

## Generate Random Data

In [16]:
import csv
import random
import json
import uuid
import io
from google.cloud import storage # GCS client library
from google.api_core import exceptions # For handling GCS exceptions

# --- Configuration ---
NUM_REVIEWS = 10000
OUTPUT_FILENAME = "mechanical_keyboard_reviews.csv"

# --- GCS Configuration ---
# !!! REPLACE WITH YOUR BUCKET NAME !!!
GCS_BUCKET_NAME =  "n25_demo"
# Desired filename within the bucket
GCS_BLOB_NAME = "input_files/" + OUTPUT_FILENAME


# --- Data Generation Options ---

# Product Name Components
PRODUCT_PREFIXES = ["Quantum", "Hyper", "Cyber", "Stellar", "Lunar", "Arctic", "Shadow", "Nova", "Mech", "Giga", "Aura", "Stealth"]
PRODUCT_CORES = ["Striker", "Weaver", "Blaze", "Forge", "Drift", "Pulse", "Core", "Shift", "Byte", "Raptor", "Phantom", "Spectre"]
PRODUCT_SUFFIXES = ["Pro", "X", "Elite", "V2", "Mini", "Max", "Omega", "Alpha", "XT", "SE", "Air", "Lite"]

# is_he Options (includes None for null)
IS_HE_OPTIONS = [True, False, 1, 0, None] # csv writer handles None as empty string

# Switch Types
SWITCH_TYPES = ["Brown", "Red", "Blue", "Black", "Clear", "Green", "Jade", "Navy", "Silver", "Silent Red", "Silent Black", "Yellow", "White", "Orange"]

# Layout Options (mix of int and string percentages)
LAYOUT_OPTIONS = [60, 65, 75, 80, 96, 100, "60%", "65%", "75%", "80%", "96%", "100%", "TKL", "Full"] # Added TKL/Full for variety

# --- Review Text Generation Components ---

# User Name Components
USER_ADJECTIVES = ["Happy", "Geeky", "Swift", "Clever", "Silent", "Angry", "Casual", "Pro", "Sleepy", "Wired"]
USER_NOUNS = ["Panda", "Coder", "Gamer", "Typist", "Ninja", "Cat", "Bear", "Fox", "Dev", "Writer"]

# Review Snippets (same as before)
OPENINGS_POS = [
    "Absolutely love this keyboard!", "Finally found my endgame keyboard.", "Incredible value for the price.",
    "Solid build quality and performance.", "Exceeded my expectations!", "A joy to type on every day.",
]
OPENINGS_NEG = [
    "Quite disappointed with this purchase.", "Not what I expected.", "Had high hopes, but it fell short.",
    "Several issues right out of the box.", "Feels cheap and unreliable.", "Regretting this buy.",
]
OPENINGS_NEU = [
    "It's an okay keyboard, does the job.", "Mixed feelings about this one.", "Decent, but has its flaws.",
    "Good for the price, but not perfect.", "An average mechanical keyboard experience.",
]
SWITCH_COMMENTS_POS = [
    "The switches feel incredibly smooth.", "Perfect tactility for typing.", "Satisfyingly clicky sound!",
    "Linear switches are great for gaming.", "Low actuation force is comfortable.", "Consistent keypresses across the board.",
]
SWITCH_COMMENTS_NEG = [
    "Switches feel scratchy.", "Too loud for an office environment.", "Keypresses feel inconsistent.",
    "Mushy feeling, lacks crispness.", "Spring ping is quite noticeable.", "Accidentally actuating keys too often.",
]
LAYOUT_COMMENTS = [
    "The compact layout saves so much desk space.", "Perfect balance between size and functionality.",
    "Really miss having a dedicated numpad.", "Took some time to get used to the layers.",
    "Standard layout makes finding keycaps easy.", "Having dedicated arrow keys is essential for me.",
]
BUILD_FEEL_COMMENTS_POS = [
    "The aluminum case feels very premium.", "Heavy and sturdy, doesn't slide around.",
    "Keycaps have a nice texture.", "Stabilizers are surprisingly good, minimal rattle.",
    "RGB lighting is vibrant and customizable.", "Braided cable is a nice touch.",
]
BUILD_FEEL_COMMENTS_NEG = [
    "The plastic body feels a bit cheap.", "Noticeable case ping.", "Keycaps feel thin and collect fingerprints.",
    "Stabilizers rattle quite a bit, especially the spacebar.", "RGB lighting is dim.", "Non-detachable cable is inconvenient.",
]
HE_COMMENTS = [ # Specific comments if is_he is True or 1
    "The Hall Effect switches are game-changing for competitive gaming.",
    "Adjustable actuation points are incredibly useful.",
    "Rapid trigger feature works flawlessly.",
    "The smoothness of these magnetic switches is unparalleled.",
    "Being able to customize the reset point is fantastic.",
]
CLOSINGS_POS = [
    "Highly recommend!", "Definitely worth the money.", "Wouldn't hesitate to buy again.", "Five stars!", "My new daily driver.",
]
CLOSINGS_NEG = [
    "Would not recommend this keyboard.", "Look elsewhere if you value quality.", "Returning this immediately.",
    "Save your money.", "Very frustrating experience.", "One star.",
]
CLOSINGS_NEU = [
    "It's decent, but keep your expectations in check.", "Good entry-level option maybe.", "You get what you pay for.",
    "Three stars - average.", "Might work for some, but not for me.",
]

# --- Helper Function to Generate Review Details (same as before) ---
def generate_review_details(is_he, switch_type, layout):
    sentiment = random.choice(["positive", "negative", "neutral"])
    details = []
    # Opening
    if sentiment == "positive": details.append(random.choice(OPENINGS_POS))
    elif sentiment == "negative": details.append(random.choice(OPENINGS_NEG))
    else: details.append(random.choice(OPENINGS_NEU))
    # Switch Comments
    if sentiment == "positive" or (sentiment == "neutral" and random.random() > 0.5):
         details.append(f"{random.choice(SWITCH_COMMENTS_POS)} The {switch_type} switches are particularly good.")
    elif sentiment == "negative" or (sentiment == "neutral" and random.random() <= 0.5):
         details.append(f"{random.choice(SWITCH_COMMENTS_NEG)} I chose the {switch_type} switches and I'm not impressed.")
    else: details.append(f"Using the {switch_type} switches takes some getting used to.")
    # Layout Comments
    details.append(f"{random.choice(LAYOUT_COMMENTS)} The {layout} layout is interesting.")
    # Build/Feel Comments
    if sentiment == "positive" or (sentiment == "neutral" and random.random() > 0.6):
         details.append(random.choice(BUILD_FEEL_COMMENTS_POS))
    elif sentiment == "negative" or (sentiment == "neutral" and random.random() <= 0.6):
         details.append(random.choice(BUILD_FEEL_COMMENTS_NEG))
    # HE Specific Comments (if applicable)
    if is_he in [True, 1] and random.random() > 0.4:
        details.append(random.choice(HE_COMMENTS))
    # Closing
    if sentiment == "positive": details.append(random.choice(CLOSINGS_POS))
    elif sentiment == "negative": details.append(random.choice(CLOSINGS_NEG))
    else: details.append(random.choice(CLOSINGS_NEU))
    # Add some variability in length
    num_sentences = random.randint(3, len(details))
    # Shuffle sentences slightly for more variety before joining
    random.shuffle(details)
    return " ".join(details[:num_sentences])

# --- Main Script ---
print(f"Generating {NUM_REVIEWS} reviews...")

rows = []
# Generate Header
header = ["product_name", "is_he", "switch_type", "layout", "review"]
rows.append(header)

# Generate Data Rows
for i in range(NUM_REVIEWS):
    # Product Name
    prod_name = f"{random.choice(PRODUCT_PREFIXES)} {random.choice(PRODUCT_CORES)} {random.choice(PRODUCT_SUFFIXES)}"
    if random.random() > 0.8: prod_name += f" {random.randint(100, 999)}"
    # is_he
    is_he_val = random.choice(IS_HE_OPTIONS)
    # Switch Type
    switch = random.choice(SWITCH_TYPES)
    # Layout
    layout_val = random.choice(LAYOUT_OPTIONS)
    # Review JSON
    username = f"{random.choice(USER_ADJECTIVES)}{random.choice(USER_NOUNS)}{random.randint(10, 999)}_{str(uuid.uuid4())[:4]}"
    review_text = generate_review_details(is_he_val, switch, layout_val)
    review_json_data = {"username": username, "review_details": review_text}
    review_json_string = json.dumps(review_json_data) # Convert dict to JSON string
    # Add row data
    rows.append([prod_name, is_he_val, switch, layout_val, review_json_string])

print(f"Finished generating {NUM_REVIEWS} reviews. Preparing for GCS upload...")

# --- Generate CSV content in memory ---
csv_data_string = None
try:
    # Use io.StringIO as an in-memory text buffer
    string_io_buffer = io.StringIO()

    # Create a CSV writer that writes to the buffer
    writer = csv.writer(string_io_buffer, quoting=csv.QUOTE_NONNUMERIC)
    writer.writerows(rows)

    # Get the CSV data as a string from the buffer
    csv_data_string = string_io_buffer.getvalue()
    string_io_buffer.close() # Close the buffer
    print("CSV data generated in memory.")

except Exception as e:
    print(f"Error generating CSV data in memory: {e}")
    exit() # Exit if we couldn't generate the data

# --- Upload to GCS ---
if csv_data_string:
    try:
        # Initialize the GCS client
        # Assumes credentials are set up via environment variable or gcloud login
        storage_client = storage.Client()

        # Get the target bucket
        bucket = storage_client.bucket(GCS_BUCKET_NAME)

        # Create a blob (file object) in the bucket
        blob = bucket.blob(GCS_BLOB_NAME)

        print(f"Uploading data to gs://{GCS_BUCKET_NAME}/{GCS_BLOB_NAME}...")

        # Upload the CSV data string
        blob.upload_from_string(
            data=csv_data_string,
            content_type='text/csv' # Set the content type for the blob
        )

        print(f"Successfully uploaded generated reviews to gs://{GCS_BUCKET_NAME}/{GCS_BLOB_NAME}")

    except exceptions.NotFound:
        print(f"Error: Bucket '{GCS_BUCKET_NAME}' not found or no write permissions.")
    except exceptions.Forbidden:
        print(f"Error: Permission denied to write to gs://{GCS_BUCKET_NAME}/{GCS_BLOB_NAME}.")
        print("Ensure your credentials have 'Storage Object Creator' or similar role on the bucket.")
    except Exception as e:
        print(f"An unexpected error occurred during GCS upload: {e}")

else:
    print("Skipping GCS upload because CSV data generation failed.")


Generating 10000 reviews...
Finished generating 10000 reviews. Preparing for GCS upload...
CSV data generated in memory.
Uploading data to gs://n25_demo/input_files/mechanical_keyboard_reviews.csv...
Successfully uploaded generated reviews to gs://n25_demo/input_files/mechanical_keyboard_reviews.csv


# Python UDF - synthetic data using Faker

### create our UDF function to use Faker

In [5]:
%%bigquery
CREATE OR REPLACE FUNCTION `sfsc-srtt-shared.next25_demo`.random_cityname()
RETURNS STRING
LANGUAGE python
OPTIONS(runtime_version="python-3.11", entry_point="random_cityname", packages=['Faker'])
AS r'''
from faker import Faker
fake = Faker()
import random

#city_names = ["San Francisco", "Cupertino", "Los Angeles", "New York"]

def random_cityname():
  return fake.city()
  #return random.choice(city_names)
''';

Query is running:   0%|          |

""


In [17]:
%%bigquery
CREATE OR REPLACE VIEW `sfsc-srtt-shared.next25_demo`.v_cleansed_keyboard_data_with_city AS
SELECT *,  `sfsc-srtt-shared.next25_demo`.random_cityname() AS city_name
FROM `sfsc-srtt-shared.next25_demo`.cleansed_keyboard_data

Query is running:   0%|          |

""


In [18]:
%%bigquery
SELECT * FROM `sfsc-srtt-shared.next25_demo`.v_cleansed_keyboard_data_with_city LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,product_name,is_he_val,switch_type,layout,review,review_details,reviewer,city_name
0,Aura Drift Elite,True,Black,,"{""username"": ""CasualNinja434_fd74"", ""review_de...",The compact layout saves so much desk space. T...,CasualNinja434_fd74,Carterfurt
1,Stealth Byte Max,True,Black,,"{""username"": ""CasualBear534_fce1"", ""review_det...",Solid build quality and performance. Being abl...,CasualBear534_fce1,Sarahfort
2,Shadow Spectre XT,True,Black,,"{""username"": ""ProBear714_508d"", ""review_detail...",Returning this immediately. Quite disappointed...,ProBear714_508d,South Jennifer
3,Aura Striker V2,True,Black,,"{""username"": ""AngryNinja85_c2f3"", ""review_deta...",Very frustrating experience. Feels cheap and u...,AngryNinja85_c2f3,Juarezburgh
4,Nova Blaze V2,True,Black,,"{""username"": ""AngryFox622_7737"", ""review_detai...",Too loud for an office environment. I chose th...,AngryFox622_7737,Lake Christopher
5,Mech Spectre Mini 339,False,Black,,"{""username"": ""SwiftDev635_d624"", ""review_detai...",Having dedicated arrow keys is essential for m...,SwiftDev635_d624,Bryanmouth
6,Nova Phantom Max 636,True,Black,,"{""username"": ""AngryDev382_0ecb"", ""review_detai...",One star. Really miss having a dedicated numpa...,AngryDev382_0ecb,Oscarland
7,Stealth Pulse Max,True,Black,,"{""username"": ""WiredNinja812_660d"", ""review_det...","Stabilizers are surprisingly good, minimal rat...",WiredNinja812_660d,Christophermouth
8,Cyber Core Air,False,Black,,"{""username"": ""CleverCoder861_9e5f"", ""review_de...",Good entry-level option maybe. Stabilizers are...,CleverCoder861_9e5f,East Robertstad
9,Giga Byte Lite,True,Black,,"{""username"": ""SwiftTypist253_b467"", ""review_de...","Heavy and sturdy, doesn't slide around. Satisf...",SwiftTypist253_b467,Collinchester


### Vector Search in BigQuery

Declare the model to be used to generate the embedding

In [20]:
%%bigquery
CREATE OR REPLACE MODEL next25_demo.embedding_model
  REMOTE WITH CONNECTION `us-central1.embedding_connection`
  OPTIONS (ENDPOINT = 'text-embedding-004');

Query is running:   0%|          |

""


Generate the embedding

In [21]:
%%bigquery
CREATE OR REPLACE TABLE `sfsc-srtt-shared.next25_demo`.cleansed_keyboard_data_with_city_embedding AS
SELECT *
FROM ML.GENERATE_EMBEDDING(
  MODEL `next25_demo.embedding_model`,
  (
   SELECT *, review_details AS content FROM `sfsc-srtt-shared.next25_demo`.v_cleansed_keyboard_data_with_city
  ),
  STRUCT(TRUE AS flatten_json_output,
    'SEMANTIC_SIMILARITY' AS task_type)
  );

Query is running:   0%|          |

""


In [22]:
%%bigquery
SELECT * FROM `sfsc-srtt-shared.next25_demo`.cleansed_keyboard_data_with_city_embedding LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,ml_generate_embedding_result,ml_generate_embedding_statistics,ml_generate_embedding_status,product_name,is_he_val,switch_type,layout,review,review_details,reviewer,city_name,content
0,"[-0.009412508457899094, -0.004547911696135998,...","{""token_count"":59,""truncated"":false}",,Arctic Blaze Mini 569,False,Black,,"{""username"": ""ProTypist75_af56"", ""review_detai...",Regretting this buy. Accidentally actuating ke...,ProTypist75_af56,Churchhaven,Regretting this buy. Accidentally actuating ke...
1,"[-0.042978476732969284, -0.02248363010585308, ...","{""token_count"":52,""truncated"":false}",,Cyber Spectre SE 917,False,Black,,"{""username"": ""ProWriter843_f980"", ""review_deta...",Exceeded my expectations! Standard layout make...,ProWriter843_f980,West Rebeccabury,Exceeded my expectations! Standard layout make...
2,"[0.014065658673644066, -0.014745556749403477, ...","{""token_count"":48,""truncated"":false}",,Quantum Core XT,False,Blue,,"{""username"": ""SwiftBear752_54b5"", ""review_deta...",Took some time to get used to the layers. The ...,SwiftBear752_54b5,West Julie,Took some time to get used to the layers. The ...
3,"[-0.041874684393405914, -0.021096929907798767,...","{""token_count"":60,""truncated"":false}",,Quantum Blaze X,False,Brown,,"{""username"": ""HappyBear942_06ba"", ""review_deta...",You get what you pay for. Keycaps feel thin an...,HappyBear942_06ba,South David,You get what you pay for. Keycaps feel thin an...
4,"[-0.0261228010058403, -0.0016519908094778657, ...","{""token_count"":48,""truncated"":false}",,Aura Core Alpha 304,False,Clear,,"{""username"": ""AngryWriter335_919f"", ""review_de...",Too loud for an office environment. I chose th...,AngryWriter335_919f,Joeland,Too loud for an office environment. I chose th...
5,"[-0.041472893208265305, 0.011380428448319435, ...","{""token_count"":45,""truncated"":false}",,Stellar Striker XT,False,Clear,,"{""username"": ""CasualCoder622_2a04"", ""review_de...",Satisfyingly clicky sound! The Clear switches ...,CasualCoder622_2a04,Perezberg,Satisfyingly clicky sound! The Clear switches ...
6,"[-0.0497964546084404, -0.0166835505515337, -0....","{""token_count"":64,""truncated"":false}",,Mech Core Elite,False,Green,,"{""username"": ""SleepyCat496_764f"", ""review_deta...",Good entry-level option maybe. Consistent keyp...,SleepyCat496_764f,New Staceybury,Good entry-level option maybe. Consistent keyp...
7,"[-0.04836918041110039, 0.0012499626027420163, ...","{""token_count"":54,""truncated"":false}",,Quantum Raptor Pro,False,Jade,,"{""username"": ""AngryWriter758_b36f"", ""review_de...",Consistent keypresses across the board. The Ja...,AngryWriter758_b36f,West Zacharyfort,Consistent keypresses across the board. The Ja...
8,"[-0.03890809044241905, -0.012311508879065514, ...","{""token_count"":54,""truncated"":false}",,Giga Weaver Alpha 438,False,Jade,,"{""username"": ""WiredCat458_c76f"", ""review_detai...","Heavy and sturdy, doesn't slide around. Having...",WiredCat458_c76f,Port Christian,"Heavy and sturdy, doesn't slide around. Having..."
9,"[-0.0038217066321521997, 0.004792314954102039,...","{""token_count"":59,""truncated"":false}",,Arctic Shift Air,False,Navy,,"{""username"": ""WiredGamer611_385b"", ""review_det...",Feels cheap and unreliable. Having dedicated a...,WiredGamer611_385b,Brianton,Feels cheap and unreliable. Having dedicated a...


####Create a SCANN index (Preview)
https://github.com/google-research/google-research/blob/master/scann/docs/algorithms.md

Best for:
- table of 200 million rows or fewer.
- frequent and large batch queries involving hundreds or more query vectors (might use brute force for small queries)

In [23]:
%%bigquery

CREATE OR REPLACE VECTOR INDEX `reviews_idx`
ON `sfsc-srtt-shared.next25_demo`.cleansed_keyboard_data_with_city_embedding(ml_generate_embedding_result)
OPTIONS(distance_type='COSINE', index_type='TREE_AH')

Query is running:   0%|          |

""


####Search

In [24]:
%%bigquery
DECLARE test_content STRING;
SET test_content = "Regretting buying a keyboard";


CREATE OR REPLACE TABLE `sfsc-srtt-shared.next25_demo`.test_content AS
SELECT text_embedding, content AS query
FROM ML.GENERATE_TEXT_EMBEDDING(
MODEL  `next25_demo.embedding_model`,
(SELECT test_content AS content));

Query is running:   0%|          |

""


In [28]:
%%bigquery

SELECT base.content, base.product_name, distance
FROM VECTOR_SEARCH(
  TABLE `sfsc-srtt-shared.next25_demo`.cleansed_keyboard_data_with_city_embedding,
  'ml_generate_embedding_result',
  TABLE `sfsc-srtt-shared.next25_demo`.test_content
)


Query is running:   0%|          |

Downloading:   0%|          |

,content,product_name,distance
0,Not what I expected. Would not recommend this ...,Mech Drift Elite 958,0.324683
1,Regretting this buy. Switches feel scratchy. I...,Mech Core Omega,0.329846
2,Would not recommend this keyboard. Regretting ...,Quantum Spectre Elite,0.331839
3,Regretting this buy. Very frustrating experien...,Stealth Phantom V2,0.333047
4,Regretting this buy. The plastic body feels a ...,Shadow Pulse Max,0.334055
5,Regretting this buy. Keycaps feel thin and col...,Stealth Core Air,0.334287
6,Regretting this buy. Having dedicated arrow ke...,Quantum Phantom Air,0.335068
7,Would not recommend this keyboard. Regretting ...,Mech Striker Mini,0.339519
8,Regretting this buy. Would not recommend this ...,Hyper Forge Air 620,0.342775
9,Regretting this buy. The plastic body feels a ...,Hyper Blaze XT,0.345810
